In [1]:
!pip install PyAthena
!pip install numpy
!pip install pandas
!pip install datetime
!pip install matplotlib
!pip install os
!pip install boto3

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [50]:
from pyathena import connect
# from pyathena.util import as_pandas
from __future__ import print_function

# Import libraries
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import boto3
from botocore.client import ClientError
# below is used to print out pretty pandas dataframes
from IPython.display import display, HTML
%matplotlib inline


s3 = boto3.resource('s3', region_name='us-east-1')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name
athena_query_results_bucket = 'my-athena-results-bucket-12343434234234324234234234'

try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
except ClientError:
    bucket = s3.create_bucket(Bucket=athena_query_results_bucket)
    print('Creating bucket '+athena_query_results_bucket)
cursor = connect(s3_staging_dir='s3://'+athena_query_results_bucket).cursor()


conn = connect(region_name="us-east-1",
               s3_staging_dir="s3://my-athena-results-bucket/",
               schema_name="your_database_name")

cursor = conn.cursor()
cursor.execute("SELECT * FROM your_table LIMIT 10")

# The Glue database name of your MIMIC-III parquet data
gluedatabase="mimiciii"

# location of the queries to generate aline specific materialized views
aline_path = './'

# location of the queries to generate materialized views from the MIMIC code repository
concepts_path = './concepts/'

Failed to execute query.
Traceback (most recent call last):
  File "/Users/pranavigupta/data_to_decisions/prevention-data/.venv/lib/python3.13/site-packages/pyathena/common.py", line 586, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/Users/pranavigupta/data_to_decisions/prevention-data/.venv/lib/python3.13/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/Users/pranavigupta/data_to_decisions/prevention-data/.venv/lib/python3.13/site-packages/tenacity/__init__.py", line 475, in __call__
    do = self.iter(retry_state=retry_state)
  File "/Users/pranavigupta/data_to_decisions/prevention-data/.venv/lib/python3.13/site-packages/tenacity/__init__.py", line 376, in iter
    result = action(retry_s

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: Unable to verify/create output bucket my-athena-results-bucket

In [45]:
import boto3
from botocore.exceptions import ClientError
from pyathena import connect
import os

# Initialize boto3 clients and session
s3 = boto3.resource('s3', region_name='us-east-1')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name

# Define your Athena results bucket
athena_query_results_bucket = 'my-athena-results-bucket-12343434234234324234234234'

# Check if the bucket exists, and create it if not
try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
    print(f"Bucket {athena_query_results_bucket} already exists.")
except ClientError:
    print(f"Bucket {athena_query_results_bucket} does not exist. Creating it now.")
    s3.create_bucket(Bucket=athena_query_results_bucket, CreateBucketConfiguration={
        'LocationConstraint': 'us-east-1'  # Ensure the region is set correctly
    })

# Connect to Athena and specify the S3 staging directory
conn = connect(region_name="us-east-1",
               s3_staging_dir=f"s3://{athena_query_results_bucket}",
               schema_name="mimiciii")  # Use the 'mimiciii' database directly

# Create cursor
cursor = conn.cursor()

# Execute query to list the available tables in the 'mimiciii' database
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("Available tables in mimiciii database:", tables) 

try:
    # Attempt to retrieve the ACL of an object
    acl = s3.ObjectAcl('mimic-iii-physionet', 'parquet/PATIENTS/')
    print(acl.grants)
except Exception as e:
    print(f"Error occurred: {e}")



# Example: Selecting data from the 'PROCEDURES_ICD' table
# You can replace 'PROCEDURES_ICD' with any table name from the list you retrieved
cursor.execute("SELECT * FROM patients LIMIT 10")

# Fetch and print results
results = cursor.fetchall()
print("Results from PATIENTS:", results)

# # Define paths for materialized views queries
# aline_path = './'
# concepts_path = './concepts/'

# # Execute queries from the specified directories for materialized views
# def execute_query_from_file(query_path):
#     with open(query_path, 'r') as file:
#         query = file.read


Bucket my-athena-results-bucket-12343434234234324234234234 already exists.
Available tables in mimiciii database: [('admissions',), ('callout',), ('caregivers',), ('chartevents',), ('cptevents',), ('d_cpt',), ('d_icd_diagnoses',), ('d_icd_procedures',), ('d_items',), ('d_labitems',), ('datetimeevents',), ('diagnoses_icd',), ('drgcodes',), ('icustays',), ('inputevents_cv',), ('inputevents_mv',), ('labevents',), ('microbiologyevents',), ('noteevents',), ('outputevents',), ('patients',), ('prescriptions',), ('procedureevents_mv',), ('procedures_icd',), ('services',), ('transfers',)]
Error occurred: An error occurred (AccessDenied) when calling the GetObjectAcl operation: Access Denied


OperationalError: com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: 5T1GZWVWCHYYBQCW; S3 Extended Request ID: VbRlRTrtDpi9t4oebhwkPJQ/66bN5QyABndq2STyiOGZz5l6eMCng6N/feKPoJvPbwqEY38EgEf+WcLc0N5Kig==; Proxy: null), S3 Extended Request ID: VbRlRTrtDpi9t4oebhwkPJQ/66bN5QyABndq2STyiOGZz5l6eMCng6N/feKPoJvPbwqEY38EgEf+WcLc0N5Kig== (Bucket: mimic-iii-physionet, Key: parquet/PATIENTS/)

In [37]:
# Load in the query from file
query='DROP TABLE IF EXISTS DATABASE.angus_sepsis;'
cursor.execute(query.replace("DATABASE", gluedatabase))
f = os.path.join(concepts_path,'sepsis/angus-awsathena.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating table \'angus_sepsis\' using {} ...'.format(f),end=' ')
cursor.execute(query.replace("DATABASE", gluedatabase))
print('done.')

# Load in the query from file
query='DROP TABLE IF EXISTS DATABASE.heightweight;'
cursor.execute(query.replace("DATABASE", gluedatabase))
f = os.path.join(concepts_path,'demographics/HeightWeightQuery-awsathena.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating table \'heightweight\' using {} ...'.format(f),end=' ')
cursor.execute(query.replace("DATABASE", gluedatabase))
print('done.')


# Load in the query from file
query='DROP TABLE IF EXISTS DATABASE.aline_vaso_flag;'
cursor.execute(query.replace("DATABASE", gluedatabase))
f = os.path.join(aline_path,'aline_vaso_flag-awsathena.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating table \'aline_vaso_flag\' using {} ...'.format(f),end=' ')
cursor.execute(query.replace("DATABASE", gluedatabase))
print('done.')


# Load in the query from file
query='DROP TABLE IF EXISTS DATABASE.ventsettings;'
cursor.execute(query.replace("DATABASE", gluedatabase))
f = os.path.join(concepts_path,'durations/ventilation-settings-awsathena.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating table \'vent_settings\' using {} ...'.format(f),end=' ')
cursor.execute(query.replace("DATABASE", gluedatabase))
print('done.')


# Load in the query from file
query='DROP TABLE IF EXISTS DATABASE.ventdurations;'
cursor.execute(query.replace("DATABASE", gluedatabase))
f = os.path.join(concepts_path,'durations/ventilation-durations-awsathena.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating table \'vent_durations\' using {} ...'.format(f),end=' ')
cursor.execute(query.replace("DATABASE", gluedatabase))
print('done.')

FileNotFoundError: [Errno 2] No such file or directory: './concepts/sepsis/angus-awsathena.sql'